## Training


In [ ]:
import numpy as np
from scipy import signal
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import joblib
import time

#For Loading Data
def load_data(data_set):
        # Load or define the labels

        file_names = {
            'training': [
                'trainLabels.npy',
                'trainAccelerometer.npy',
                'trainGravity.npy',
                'trainGyroscope.npy',
                'trainJinsAccelerometer.npy',
                'trainJinsGyroscope.npy',
                'trainLinearAcceleration.npy',
                'trainMagnetometer.npy',
                'trainMSAccelerometer.npy',
                'trainMSGyroscope.npy'
            ],
            'testing': [
                'testLabels.npy',
                'testAccelerometer.npy',
                'testGravity.npy',
                'testGyroscope.npy',
                'testJinsAccelerometer.npy',
                'testJinsGyroscope.npy',
                'testLinearAcceleration.npy',
                'testMagnetometer.npy',
                'testMSAccelerometer.npy',
                'testMSGyroscope.npy'
            ]
        }
        data = []
        for file_name in file_names[data_set]:
            file_path = data_dir + data_set + '/' + file_name
            loaded_data = np.load(file_path)
            data.append(loaded_data)
            print(loaded_data.shape, "\t", file_name)
        return data

In [ ]:
#Main Class for training and testing
class ActivityMonitor:
    def __init__(self, train_data, test_data, train_labels, test_labels):
        self.train_data = train_data
        self.test_data = test_data
        self.train_labels = train_labels
        self.test_labels = test_labels
        self.path_save_model = "ActivityMonitoring/TrainedModels/"
        
    def normalize_data(self, data):
        reshaped_data = np.reshape(data, (-1, data.shape[2]))
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
        normalized_data = (data - mean) / std
        normalized_data = np.reshape(normalized_data, data.shape)
        return normalized_data

    def resample_data(self, data, target_frequency):
        current_frequency = data.shape[1] / 4.0
        resampled_data = signal.resample(data, int(data.shape[1] * target_frequency / current_frequency), axis=1)
        return resampled_data

    def extract_features(self, data):
        mean = np.mean(data, axis=1)
        std = np.std(data, axis=1)
        min_val = np.min(data, axis=1)
        max_val = np.max(data, axis=1)
        median = np.median(data, axis=1)
        kurt = kurtosis(data, axis=1)
        skewness = skew(data, axis=1)
        features = np.concatenate([mean, std, min_val, max_val, median, kurt, skewness], axis=1)
        return features

    def process_data(self, data):
        sensor_frequencies = {
            1: 200,  # Accelerometer
            2: 200,  # Gravity
            3: 200,  # Gyroscope
            4: 20,   # JinsAccelerometer
            5: 20,   # JinsGyroscope
            6: 200,  # LinearAcceleration
            7: 50,   # Magnetometer
            8: 67,   # MSAccelerometer
            9: 67    # MSGyroscope
        }
        processed_data = []
        for i, d in enumerate(data[1:], start=1):
            if i in sensor_frequencies:
                target_frequency = 200
                resampled_data = self.resample_data(d, sensor_frequencies[i])
                normalized_data = self.normalize_data(resampled_data)
                extracted_features = self.extract_features(normalized_data)
                processed_data.append(extracted_features)
            else:
                processed_data.append(d)
        processed_data.insert(0, data[0])
        return processed_data

    def preprocess_data(self):
        normalized_train = self.process_data(self.train_data)
        normalized_test = self.process_data(self.test_data)
        train_labels = normalized_train[0]
        self.train_data = np.concatenate(normalized_train[1:], axis=1).reshape(len(train_labels), -1)
        test_labels = normalized_test[0]
        self.test_data = np.concatenate(normalized_test[1:], axis=1).reshape(len(test_labels), -1)
#------------------------------------------------------------------------------------------------------------------------------------------
    def calculate_evaluation_metrics(self, true_labels, predicted_labels, set_name):
        acc = accuracy_score(true_labels, predicted_labels)
        cm = confusion_matrix(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        print("\n")
        print(set_name, "Accuracy:", acc)
        print(set_name, "Precision:", precision)
        print(set_name, "F1 Score:", f1)
        print(set_name, "Recall:", recall)
        print(set_name, "Confusion Matrix:\n", cm)
        return acc, cm, f1, precision, recall
#------------------------------------------------------------------------------------------------------------------------------------------ 
    #Random Forest Classifier
    def random_forest_classifier(self):
        classifier = RandomForestClassifier()
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val)
        joblib.dump(classifier,self.path_save_model + "random_forest_classifier.joblib")
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
        
    #Finding Hyperparameters Random Forest Grid
    # def random_forest_grid_search(self):
    #     param_grid = {
    #         'n_estimators': [100, 200, 300],
    #         'max_depth': [None, 5, 10],
    #         'min_samples_split': [2, 5, 10]
    #     }
    #     classifier = RandomForestClassifier()
    #     grid_search = GridSearchCV(classifier, param_grid, cv=5)
    #     grid_search.fit(self.train_data, self.train_labels)
    #     best_classifier = grid_search.best_estimator_
    #     print("Selected Hyperparameters:", best_classifier.get_params(), "\n\n")
    #     cv_scores = cross_val_score(best_classifier, self.train_data, self.train_labels, cv=5)
    #     print("Cross-validation scores:", cv_scores)
    #     print("Mean CV score:", np.mean(cv_scores))
    #     X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    #     best_classifier.fit(self.train_data, self.train_labels)
    #     val_predictions = best_classifier.predict(self.X_val)
    #     joblib.dump(best_classifier,self.path_save_model + "random_forest_grid_search.joblib")
    #     val_metrics = self.calculate_evaluation_metrics(self.y_val, val_predictions, "Validation")
    #     test_predictions = best_classifier.predict(self.test_data)
    #     test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
    #     return best_classifier
        
    #Random Forest Grid with trained hyperparameters
    def random_forest_grid_search_hyperparameters(self):
        classifier = RandomForestClassifier(
            bootstrap=True,
            ccp_alpha=0.0,
            class_weight=None,
            criterion='gini',
            max_depth=None,
            # max_features='auto',
            max_leaf_nodes=None,
            max_samples=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1,
            min_samples_split=2,
            min_weight_fraction_leaf=0.0,
            n_estimators=300,
            n_jobs=None,
            oob_score=False,
            random_state=None,
            verbose=0,
            warm_start=False
        )
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        # Fit the classifier on the training data
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val) 
        # Save model
        joblib.dump(classifier,self.path_save_model + "random_forest_grid_search_hyperparameters.joblib")
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
#------------------------------------------------------------------------------------------------------------------------------------------
    #SVM Classifier
    def svm_classifier(self):
        classifier = SVC()
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val)
        joblib.dump(classifier,self.path_save_model + "svm_classifier.joblib")
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
    
    #Finding Hyperparameters SVM Grid
    # def svm_grid_search(self):
    #     param_grid = {
    #         'C': [0.1, 1, 10],
    #         'kernel': ['linear', 'rbf', 'sigmoid'],
    #         'gamma': [0.1, 1, 'scale']
    #     }
    #     classifier = SVC()
    #     grid_search = GridSearchCV(classifier, param_grid, cv=5)
    #     X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    # 
    #     grid_search.fit(self.train_data, self.train_labels)
    #     best_classifier = grid_search.best_estimator_
    #     print("Selected Hyperparameters:", best_classifier.get_params(), "\n\n")
    #     cv_scores = cross_val_score(best_classifier, self.train_data, self.train_labels, cv=5)
    #     print("Cross-validation scores:", cv_scores)
    #     print("Mean CV score:", np.mean(cv_scores))
    #     best_classifier.fit(self.train_data, self.train_labels)
    #     val_predictions = best_classifier.predict(self.X_val)
    #     joblib.dump(best_classifier,"ActivityMonitoring/svm_grid_search.joblib")
    #     val_metrics = self.calculate_evaluation_metrics(self.y_val, val_predictions, "Validation")
    #     test_predictions = best_classifier.predict(self.test_data)
    #     test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
    #     return best_classifier
    
    #SVM Grid with trained hyperparameters
    def svm_grid_search_hyperparameters(self):
        classifier = SVC(
            C=0.1,
            break_ties=False,
            cache_size=200,
            class_weight=None,
            coef0=0.0,
            decision_function_shape='ovr',
            degree=3,
            gamma=0.1,
            kernel='linear',
            max_iter=-1,
            probability=False,
            random_state=None,
            shrinking=True,
            tol=0.001,
            verbose=False,
        )
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        # Fit the classifier on the training data
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val) 
        # Save model
        joblib.dump(classifier,self.path_save_model + "svm_grid_search_hyperparameters.joblib")
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
        
# Example usage:
if __name__ == "__main__":

    data_dir = 'ActivityMonitoring/'
    train_data = load_data("training")
    test_data = load_data("testing")
    train_labels = np.load(data_dir + 'training/trainLabels.npy')
    test_labels = np.load(data_dir + 'testing/testLabels.npy')

    data_loader = ActivityMonitor(train_data, test_data, train_labels, test_labels)
    data_loader.preprocess_data()

    # Random Forest Classifier
    random_forest_clf = data_loader.random_forest_classifier()
    random_forest_clf_grid = data_loader.random_forest_grid_search_hyperparameters()

    # SVM Classifier
    svm_clf = data_loader.svm_classifier()
    svm_grid_clf = data_loader.svm_grid_search_hyperparameters()
    
    
    

## Testing


In [ ]:
import numpy as np
import joblib

def predict_activity(inp, model_no):
    path_save_model = "ActivityMonitoring/TrainedModels/"
    activity_labels = {
        0: "Bring",
        1: "Clean Floor",
        2: "Clean Surface",
        3: "Close Big Box",
        4: "Close Door",
        5: "Close Drawer",
        6: "Close Lid By Rotate",
        7: "Close Other Lid",
        8: "Close Small Box",
        9: "Close Tap Water",
        10: "Drink",
        11: "Dry Off Hands",
        12: "Dry Off Hands By Shake",
        13: "Eat Small",
        14: "Gargle",
        15: "Getting Up",
        16: "Hang",
        17: "Lying Down",
        18: "Open Bag",
        19: "Open Big Box",
        20: "Open Door",
        21: "Open Drawer",
        22: "Open Lid By Rotate",
        23: "Open Other Lid",
        24: "Open Small Box",
        25: "Open Tap Water",
        26: "Plug In",
        27: "Press By Grasp",
        28: "Press From Top",
        29: "Press Switch",
        30: "Put From Bottle",
        31: "Put From Tap Water",
        32: "Put High Position",
        33: "Put On Floor",
        34: "Read",
        35: "Rotate",
        36: "Rub Hands",
        37: "Scoop And Put",
        38: "Sitting Down",
        39: "Squatting Down",
        40: "Standing Up",
        41: "Stand Up From Squatting",
        42: "Take From Floor",
        43: "Take From High Position",
        44: "Take Off Jacket",
        45: "Take Out",
        46: "Talk By Telephone",
        47: "Throw Out",
        48: "Throw Out Water",
        49: "Touch Smartphone Screen",
        50: "Type",
        51: "Unhang",
        52: "Unplug",
        53: "Wear Jacket",
        54: "Write"
    }
    
    classifier = joblib.load(path_save_model + "random_forest_classifier.joblib")
    
    if(model_no == "Random Forest"):
        classifier = joblib.load(path_save_model + "random_forest_classifier.joblib")    
    elif(model_no == "Random Forest Grid"):
        classifier = joblib.load(path_save_model + "random_forest_grid_search_hyperparameters.joblib")
    elif(model_no == "SVM"):
        classifier = joblib.load(path_save_model + "svm_classifier.joblib")
    elif(model_no == "SVM Grid"):
        classifier = joblib.load(path_save_model + "svm_grid_search_hyperparameters.joblib")
    
    #arr = [data_loader.test_data[int(inp), :]]
    true_label = inp[0]
    arr = inp[1:]

    np_arr = np.array(arr)
    #print(np_arr.shape)
    
    prediction = classifier.predict(np_arr.reshape(1, -1))
    
    #print(prediction[0])
        
    activity = activity_labels.get(prediction[0])
    
    print(activity)
    true_label_concat = "Label = " + str(true_label) + "\tActivity = " + activity_labels.get(true_label)
    predicted_label_concat = "Label = " + str(prediction[0]) + "\tActivity = " + activity
    
    return true_label_concat, predicted_label_concat

In [ ]:
import gradio as gr

def predict_wrapper(csv_file_path, model):
    data = np.genfromtxt(csv_file_path, delimiter=',')    
    return predict_activity(data, model)

activity_prediction = gr.Interface(
    fn=predict_wrapper,  # Pass your function
    inputs=[
        gr.File(label="Upload CSV File"),
        #gr.Textbox(type="text", label="Model No", placeholder="Enter model to test")
        gr.Radio(choices=["Random Forest", "Random Forest Grid", "SVM", "SVM Grid"], value="Random Forest")
    ],
    outputs=[
        gr.Textbox(type="text", label="True Label"),
        gr.Textbox(type="text", label="Prediction")
    ]  # Output component
)
activity_prediction.launch()